In [ ]:
pip install numpy==1.25.0 pandas matplotlib tensorflow tensorflow_federated diffprivlib syft flask

In [ ]:
# Load and Preprocess the Dataset
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler
from diffprivlib.mechanisms import Laplace

# Load the dataset; note that the file uses ';' as the delimiter.
df = pd.read_csv("cardio_train.csv", sep=';')

# Drop the 'id' column if present
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Display dataset shape and first few rows
print("Dataset shape:", df.shape)
print("Dataset preview:")
print(df.head())

# Apply differential privacy: add Laplace noise to each numeric feature (except the target 'cardio')
def apply_differential_privacy(data, epsilon):
    dp_data = data.copy()
    for col in data.columns:
        if col != 'cardio':  # Skip the target column
            # Calculate sensitivity as the range of the column values
            sensitivity = data[col].max() - data[col].min()
            # Add Laplace noise (you can adjust epsilon to change the noise magnitude)
            noise = Laplace(epsilon=epsilon, sensitivity=sensitivity).randomise(0)
            dp_data[col] = dp_data[col] + noise
    return dp_data

# Apply differential privacy to the entire dataset
df_dp = apply_differential_privacy(df, epsilon=0.5)

# Split features and labels
X = df_dp.drop(columns=['cardio']).values.astype('float32')
y = df_dp['cardio'].values.astype('float32')

# Standardize features for better model performance
scaler = StandardScaler()
X = scaler.fit_transform(X)

print("Preprocessing complete. Features shape:", X.shape)


Dataset shape: (70000, 12)
Dataset preview:
     age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0  18393       2     168    62.0    110     80            1     1      0   
1  20228       1     156    85.0    140     90            3     1      0   
2  18857       1     165    64.0    130     70            3     1      0   
3  17623       2     169    82.0    150    100            1     1      0   
4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  
0     0       1       0  
1     0       1       1  
2     0       0       1  
3     0       1       1  
4     0       0       0  
Preprocessing complete. Features shape: (70000, 11)


In [ ]:
print(df['cardio'].value_counts())


cardio
0    35021
1    34979
Name: count, dtype: int64


In [ ]:
# Simulate Federated Clients and Create TensorFlow Datasets
# Function to partition the data into N clients
def create_clients(X, y, num_clients):
    client_data = []
    data_size = len(X) // num_clients
    for i in range(num_clients):
        start = i * data_size
        # For the last client, include all remaining data
        end = (i + 1) * data_size if i < num_clients - 1 else len(X)
        client_data.append({'X': X[start:end], 'y': y[start:end]})
    return client_data

# Create 10 clients
clients = create_clients(X, y, num_clients=10)

# Convert each client's data into a tf.data.Dataset
def create_tf_dataset(client):
    dataset = tf.data.Dataset.from_tensor_slices((client['X'], client['y']))
    return dataset.shuffle(buffer_size=len(client['X'])).batch(32)

client_datasets = [create_tf_dataset(client) for client in clients]

print("Created", len(client_datasets), "client datasets for federated learning.")


Created 10 client datasets for federated learning.


In [ ]:
#  Define and Wrap the Model for Federated Learning
# Define a simple Keras model; adjust architecture as needed
def create_keras_model():
    # model = tf.keras.Sequential([
    #     tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    #     tf.keras.layers.Dense(32, activation='relu'),
    #     tf.keras.layers.Dense(1, activation='sigmoid')
    # ])
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Wrap the Keras model using TFF's from_keras_model
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

print("Model function defined for federated learning.")

Model function defined for federated learning.


In [ ]:
# Set Up and Run Federated Training
# Define client and server optimizers
client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.0001)
server_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.0001)

# Build the federated averaging process
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

# Initialize the federated learning process state
state = iterative_process.initialize()
print("Federated learning model initialized.")

# Run federated training over several rounds
NUM_ROUNDS = 100
for round_num in range(1, NUM_ROUNDS + 1):
    # Optionally, sample a subset of clients each round; here we use all clients
    state, metrics = iterative_process.next(state, client_datasets)
    print(f"Round {round_num}, Metrics: {metrics}")

print("Federated training complete.")


Federated learning model initialized.
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.53462857), ('loss', 0.84582096), ('num_examples', 70000), ('num_batches', 2190)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5393571), ('loss', 0.84299016), ('num_examples', 70000), ('num_batches', 2190)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5403714), ('loss', 0.84240687), ('num_examples', 70000), ('num_batches', 2190)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())

In [ ]:
# Create a centralized dataset for evaluation from the entire preprocessed data
def create_full_tf_dataset(X, y):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.batch(8)

full_dataset = create_full_tf_dataset(X, y)

# For evaluation, create and compile a fresh Keras model
trained_model = create_keras_model()
trained_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# (Optional) Fine-tune the model on the full dataset for a few epochs
history = trained_model.fit(full_dataset, epochs=30, verbose=1)

loss, accuracy = trained_model.evaluate(full_dataset)
print(f"Centralized Evaluation Accuracy: {accuracy:.4f}")


Epoch 1/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5783 - accuracy: 0.7152
Epoch 2/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5519 - accuracy: 0.7296
Epoch 3/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5478 - accuracy: 0.7322
Epoch 4/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5454 - accuracy: 0.7334
Epoch 5/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5458 - accuracy: 0.7343
Epoch 6/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5454 - accuracy: 0.7351
Epoch 7/30
8750/8750 [==============================] - 16s 2ms/step - loss: 0.5440 - accuracy: 0.7354
Epoch 8/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5426 - accuracy: 0.7356
Epoch 9/30
8750/8750 [==============================] - 17s 2ms/step - loss: 0.5432 - accuracy: 0.7361
Epoch 10/30
8750/8750 [==============================] - 17s 2ms/step - l

KeyboardInterrupt: 